In [1]:
# Build a Dashboard Application with Plotly Dash
# In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.
# This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart.
# You will be guided to build this dashboard application via the following tasks:
# TASK 1: Add a Launch Site Drop-down Input Component
# TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
# TASK 3: Add a Range Slider to Select Payload
# TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
# Note: Please take screenshots of the Dashboard and save them.
# Further upload your notebook to github. The github URL and the screenshots are later required in the presentation slides.


In [2]:
# Import required libraries
import requests
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import webbrowser
import numpy as np




In [3]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

spacex_df.head()


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
# Create a dash application
app = dash.Dash(__name__)

In [5]:
# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # TASK 1: Add a dropdown list to enable Launch Site selection
    # The default select value is for ALL sites
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                 ],
                 value='ALL',
                 placeholder="Select a Launch Site here",
                 searchable=True
                 ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    # If a specific launch site was selected, show the Success vs. Failed counts for the site
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),


    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(id='payload-slider',
                    min=min_payload,
                    max=max_payload,
                    step=1000,  # You may adjust the step as necessary
                    marks={i: f'{int(i)} Kg' for i in np.linspace(min_payload, max_payload, num=6)},  # Generates 6 evenly spaced marks
                    value=[min_payload, max_payload]),  # Default range set to min and max of payload
    
    
    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])




In [6]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Summarize the count of successful launches for all sites
        filtered_df = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Successful Launches by Site')
    else:
        # Display success vs. failed counts for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        success_fail = filtered_df['class'].value_counts().reset_index()
        success_fail.columns = ['Outcome', 'Count']
        success_fail['Outcome'] = success_fail['Outcome'].replace({1: 'Success', 0: 'Fail'})
        fig = px.pie(success_fail, values='Count', names='Outcome', title=f'Success vs. Fail at {selected_site}')

    return fig




In [7]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    # Filter the DataFrame based on the selected site if it is not 'ALL'
    if selected_site != 'ALL':
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
    else:
        filtered_df = spacex_df

    # Further filter the DataFrame based on the selected payload range
    mask = (filtered_df['Payload Mass (kg)'] >= payload_range[0]) & \
           (filtered_df['Payload Mass (kg)'] <= payload_range[1])
    filtered_df = filtered_df[mask]

    # Create a scatter plot to show the correlation between payload and launch success
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                     color='Booster Version Category',
                     title='Correlation between Payload and Success for ' + selected_site + ' Landing Sites',
                     labels={'class': 'Launch Success'})
    return fig




In [8]:
# # Run the app
   
# if __name__ == '__main__':
#     app.run_server(port=8080)  # Change 8080 to another number if this port is also in use

In [9]:
#Run the app
def open_browser():
    webbrowser.open('http://localhost:5050')

if __name__ == '__main__':
    # Open the web browser before running the app
    open_browser()
    # Run the app on a different port, e.g., 5050
    app.run_server(port=5050)
    
    
    